In [2]:
#importing all the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from nltk.corpus import stopwords

In [3]:
df_test=pd.read_csv(r'fake-news-detection-data\test.csv')
df_train=pd.read_csv(r'fake-news-detection-data\train.csv')


In [4]:
df_train.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [5]:
df_test.head(10)

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
5,20805,Trump is USA's antique hero. Clinton will be n...,NaN,Trump is USA's antique hero. Clinton will be n...
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori..."
7,20807,Weekly Featured Profile – Randy Shannon,Trevor Loudon,You are here: Home / *Articles of the Bound* /...
8,20808,Urban Population Booms Will Make Climate Chang...,NaN,Urban Population Booms Will Make Climate Chang...
9,20809,NaN,cognitive dissident,don't we have the receipt?


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
#removing null values
df_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df_train=df_train.fillna(" ")

In [9]:
df_train.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20800 non-null  object
 2   author  20800 non-null  object
 3   text    20800 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [11]:
#removing unwanted columns
df_train.drop(columns = 'id',axis=1,inplace=True)

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20800 non-null  object
 1   author  20800 non-null  object
 2   text    20800 non-null  object
 3   label   20800 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 650.1+ KB


In [13]:
#merging columns
df_train['news']=df_train['author']+':'+df_train['title']

In [14]:
df_train.head(10)

,title,author,text,label,news
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus:House Dem Aide: We Didn’t Even S...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn:FLYNN: Hillary Clinton, Big Wo..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com:Why the Truth Might Get You...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss:15 Civilians Killed In Single ...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy:Iranian woman jailed for fictio...
5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0,Daniel Nussbaum:Jackie Mason: Hollywood Would ...
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,,Ever wonder how Britain’s most iconic pop pian...,1,:Life: Life Of Luxury: Elton John’s 6 Favorit...
7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0,Alissa J. Rubin:Benoît Hamon Wins French Socia...
8,Excerpts From a Draft Script for Donald Trump’...,,Donald J. Trump is scheduled to make a highly ...,0,:Excerpts From a Draft Script for Donald Trum...
9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0,Megan Twohey and Scott Shane:A Back-Channel Pl...


In [15]:
# df_train.drop(columns = ['text','author'],axis=1,inplace=True)

In [16]:
df_train.head(10)

,title,author,text,label,news
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus:House Dem Aide: We Didn’t Even S...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn:FLYNN: Hillary Clinton, Big Wo..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com:Why the Truth Might Get You...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss:15 Civilians Killed In Single ...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy:Iranian woman jailed for fictio...
5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0,Daniel Nussbaum:Jackie Mason: Hollywood Would ...
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,,Ever wonder how Britain’s most iconic pop pian...,1,:Life: Life Of Luxury: Elton John’s 6 Favorit...
7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0,Alissa J. Rubin:Benoît Hamon Wins French Socia...
8,Excerpts From a Draft Script for Donald Trump’...,,Donald J. Trump is scheduled to make a highly ...,0,:Excerpts From a Draft Script for Donald Trum...
9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0,Megan Twohey and Scott Shane:A Back-Channel Pl...


In [17]:
# Stemming

from nltk.stem.porter import PorterStemmer


In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
'''stemmer = PorterStemmer()'''

'stemmer = PorterStemmer()'

In [20]:
'''def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content'''

"def stemming(content):\n    stemmed_content = re.sub('[^a-zA-Z]',' ',content)\n    stemmed_content = stemmed_content.lower()\n    stemmed_content = stemmed_content.split()\n    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]\n    stemmed_content = ' '.join(stemmed_content)\n    return stemmed_content"

In [21]:
'''df_train['news'] = df_train['news'].apply(stemming)'''

"df_train['news'] = df_train['news'].apply(stemming)"

In [22]:
X = df_train['news'].values
Y = df_train['label'].values

In [23]:
type(df_train['news'].values)

numpy.ndarray

In [24]:
print(X)

['Darrell Lucus:House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'
 'Daniel J. Flynn:FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
 'Consortiumnews.com:Why the Truth Might Get You Fired' ...
 'Michael J. de la Merced and Rachel Abrams:Macy’s Is Said to Receive Takeover Approach by Hudson’s Bay - The New York Times'
 'Alex Ansary:NATO, Russia To Hold Parallel Exercises In Balkans'
 'David Swanson:What Keeps the F-35 Alive']


In [25]:
print(Y)

[1 0 1 ... 0 1 1]


In [26]:
X.shape

(20800,)

In [27]:
Y.shape

(20800,)

In [28]:
'''from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lem_sent = [lemmatizer.lemmatize(words_sent) for words_sent in sent]
print(lem_sent)'''

'from nltk.stem.wordnet import WordNetLemmatizer\nlemmatizer = WordNetLemmatizer()\nlem_sent = [lemmatizer.lemmatize(words_sent) for words_sent in sent]\nprint(lem_sent)'

In [29]:
'''#vectorising
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
vectorizer.fit(X)'''

'#vectorising\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nvectorizer=TfidfVectorizer()\nvectorizer.fit(X)'

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

vectorizer.fit(X)

CountVectorizer()

In [31]:
type(X)

numpy.ndarray

In [32]:
X=vectorizer.transform(X)

In [33]:
print(X)

  (0, 904)	1
  (0, 3862)	1
  (0, 4507)	1
  (0, 5508)	1
  (0, 5800)	1
  (0, 6145)	1
  (0, 7574)	1
  (0, 10387)	1
  (0, 11307)	1
  (0, 11409)	1
  (0, 12528)	1
  (0, 12902)	1
  (0, 19171)	1
  (0, 22289)	1
  (0, 22649)	1
  (0, 23355)	1
  (1, 2544)	1
  (1, 3075)	1
  (1, 3509)	1
  (1, 4298)	1
  (1, 5469)	1
  (1, 8420)	2
  (1, 10134)	1
  (1, 15149)	1
  (1, 23748)	1
  :	:
  (20797, 17253)	1
  (20797, 17550)	1
  (20797, 18720)	1
  (20797, 21266)	1
  (20797, 21564)	1
  (20797, 21736)	1
  (20797, 21775)	1
  (20797, 23989)	1
  (20798, 1009)	1
  (20798, 1324)	1
  (20798, 2068)	1
  (20798, 7676)	1
  (20798, 10228)	1
  (20798, 10764)	1
  (20798, 14500)	1
  (20798, 15626)	1
  (20798, 18626)	1
  (20798, 21775)	1
  (20799, 270)	1
  (20799, 1043)	1
  (20799, 5537)	1
  (20799, 11815)	1
  (20799, 21101)	1
  (20799, 21564)	1
  (20799, 23493)	1


In [34]:
#splitting the data to train and test
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [35]:
#model - logistic regression
from sklearn.linear_model import LogisticRegression
regressor=LogisticRegression()
regressor.fit(X_train,Y_train)

LogisticRegression()

In [36]:
lr_predict=regressor.predict(X_test)

In [37]:
regressor.score(X_test,Y_test)

0.9913461538461539

In [38]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [39]:
accuracy=accuracy_score(lr_predict,Y_test)

In [40]:
accuracy


0.9913461538461539

In [41]:
print(confusion_matrix(Y_test,lr_predict))

[[2040   30]
 [   6 2084]]


In [42]:
X_new = X_test[150]


prediction = regressor.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')


[0]
The news is Real


In [43]:
#save the model
import pickle

In [45]:
import os
os.mkdir("./models")
filename='./models/logreg.sav'
pickle.dump(regressor,open(filename,'wb'))

In [62]:
loaded_model=pickle.load(open('./models/logreg.sav','rb'))

In [70]:
# saving the vectorizer
os.mkdir("./vectorizer")
filename='./vectorizer/author_title.sav'
pickle.dump(vectorizer,open(filename,'wb'))

In [71]:
loaded_vectorizer = pickle.load(open('./vectorizer/author_title.sav','rb'))

In [72]:
X_new = X_test[150]
print(type(X_new))
prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

<class 'scipy.sparse.csr.csr_matrix'>
[0]
The news is Real


In [77]:
X_new=['Biden, a Democrat from Delaware who previously served as vice president under Barack Obama, took office following his victory in the 2020 presidential election over Republican incumbent president Donald Trump. Upon his inauguration, he became the oldest president in American history.']

In [78]:
type(X_new)

list

In [79]:

X_new=loaded_vectorizer.transform(X_new)
type(X_new)

scipy.sparse.csr.csr_matrix

In [80]:
prediction = loaded_model.predict(X_new)

In [81]:
if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

The news is Real


In [82]:
fake_test ="Massachusetts Cop’s Wife Busted for Pinning Fake Home-Invasion Robbery on Black Lives Matter Source: PINAC The wife of a Massachusetts cop is facing charges for filing a false police report after she concocted a story about thieves ransacking her home, stealing $10,000 in jewelry and then spray-painting her house with Black Lives Matter to pin it on black people. Maria Daly, wife of Millbury K-9 cop Daniel Daly, took to social media about her dreamt-up victimization after she filed a police report on October 17. “We woke up to not only our house being robbed while we were sleeping, but to see this hatred for no reason,” she posted, according to the Boston Herald. “ If you would of [sic] asked me yesterday about this blue lives and black lives matter issue my response would of [sic] been very positive [sic],” the now private Facebook account alleged. “Today on the other hand I have so much anger and hate that I don’t like myself. This is what we have to deal with these days and it makes me sick that this is what was on the side of my house.” Maria Daly called police to report someone had robbed her home while she was sleeping and then spray-painted her home with BLM for Black Lives Matter then posted about it on social media. Millbury Police Chief Donald Desourcy told the Herald that Daly called police to report an early morning break-in October 17, claiming someone had made off with thousands of dollars worth of valuables and spray-painting “BLM” on the outside of her house. But Chief Desourcy stated that as the investigation unfolded, something didn’t feel “quite right” and Daly ended up admitting she fabricated the whole thing, telling the officers her valuables had already been recovered. “It was pretty obvious. The officers did their due diligence and followed through with the investigation that we had,” he told CBS Boston . “We came to the conclusion it was all fabricated. There was no intruder, there was no burglary.” The chief said the hoax was likely motivated by the couple’s financial troubles and that he has empathy for the family. “I’m very familiar with her and it’s an unfortunate set of circumstances that have taken place.” Daly’s neighbors said it wasn’t a very smart thing to do for a woman who is married to a cop. “She must have tagged the place herself,” said one neighbor. “I don’t know why you’d do that, if you’re gonna stage a robbery, I mean really come on, you’re a cop’s wife. You should know better.” In addition to charges for filing a false police report, Daly also faces a charge for misleading a police investigation. Her cases will be heard at Worcester District Court after she is summoned. Daly’s husband, Daniel Daly was not involved or charged in relation to the hoax, according to police. Share This Article..."
inp = loaded_vectorizer.transform([fake_test])

prediction = loaded_model.predict(inp)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

The news is Fake


In [83]:
## training on author + text data
df_train['newstext'] = df_train['author'] + ' : ' + df_train['text']

In [90]:
X_text, Y_text = df_train['newstext'].values, df_train['label'].values

In [91]:
vectorizer_text = CountVectorizer()

In [97]:
X_text = vectorizer_text.fit_transform(X_text)

In [98]:
X_text_train, X_text_test, Y_text_train, Y_text_test = train_test_split(X_text, Y_text, test_size=0.2)

In [99]:
logreg_text = LogisticRegression()

In [100]:
logreg_text.fit(X_text_train, Y_text_train)

c:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [101]:
logreg_text.score(X_text_test, Y_text_test)

0.9665865384615384

In [102]:
# export new model
pickle.dump(logreg_text, open('./models/logreg_text.sav','wb'))

In [104]:
#export new vectorizer
pickle.dump(vectorizer_text, open('./vectorizer/author_text.sav','wb'))

In [106]:
# now load the saved models for testing
loaded_text_model = pickle.load(open('./models/logreg_text.sav','rb'))
loaded_text_vectorizer = pickle.load(open('./vectorizer/author_text.sav','rb'))

In [107]:
loaded_text_model.predict(X_text_test[120]) == Y_text_test[120]

array([ True])